# TPz lite

_Authors: Andreia Dourado, Bruno Moraes_

_Adapted from Sam Schmidt example notebook: https://github.com/LSSTDESC/rail_tpz ._


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rail
import qp
from rail.core.data import TableHandle, PqHandle, ModelHandle, QPHandle, DataHandle, Hdf5Handle
from rail.core.data import TableHandle
from rail.core.stage import RailStage
import os
import tables_io

DS = RailStage.data_store
DS.__class__.allow_overwrite = True

from rail.estimation.algos.tpz_lite import TPZliteInformer
from rail.estimation.algos.tpz_lite import TPZliteEstimator

In [ ]:
trainfile = '/home/andreia.dourado/ic-photoz/andreia_dourado/cosmodc2_Y1_a_testing_200k.hdf5'
testfile =  '/home/andreia.dourado/ic-photoz/andreia_dourado/cosmodc2_Y1_a_testing_200k.hdf5'

Creating the training_data file:

In [ ]:
training_data = DS.read_file("training_data", TableHandle, trainfile)

In [ ]:
print(training_data.data)

In [ ]:
#len(training_data.data['ra'])

Creating the test_data file:

In [ ]:
test_data = DS.read_file("test_data", TableHandle, testfile)

In [ ]:
#len(test_data.data['ra'])

#### 2. Inform method

Parametros:

In [ ]:
bands = ["g", "i", "r", "u", "y", "z"]
new_err_dict = {}
train_atts = []
new_mag_limits={}
for band in bands:
    train_atts.append(f"mag_{band}_lsst")
    new_err_dict[f"mag_{band}_lsst"] = f"mag_err_{band}_lsst"
    new_mag_limits[f"mag_{band}_lsst"] = max(training_data.data['photometry'][f"mag_{band}_lsst"])
# redshift is also an attribute used in the training, but it does not have an associated
# error its entry in the err_dict should be set to "None"
new_err_dict["redshift"] = None

print(new_err_dict)|
print(train_atts)
print(new_mag_limits)

In [ ]:
new_err_dict.items()

In [ ]:
tpz_dict = dict(zmin=min(training_data.data['photometry']['redshift']), 
                zmax=max(training_data.data['photometry']['redshift']), 
                nzbins=301, 
                mag_limits=new_mag_limits,
                bands=train_atts,
                hdf5_groupname='photometry',
                use_atts=train_atts,
                err_dict=new_err_dict,
                nrandom=20, 
                ntrees=5,
                tree_strategy='sklearn') 

Run:

In [ ]:
pz_train = TPZliteInformer.make_stage(name='inform_TPZ', model='estimator_magerr.pkl', **tpz_dict)

In [ ]:
%%time
pz_train.inform(training_data)

#### 3.3 Estimate stage

Parametros:

In [ ]:
test_dict = dict(hdf5_groupname=None,
                test_err_dict=new_err_dict,
                mag_limits=new_mag_limits)
test_dict

Ler um .pkl file prévio:

In [ ]:
#import pickle

In [ ]:
#model = pickle.load(open('/home/andreia.dourado/ic-photoz/andreia_dourado/estimator_tpz_run_102trees.pkl','rb'))

Ler o .pkl file gerado no inform neste notebook:

In [ ]:
model = pz_train.get_handle('model')

In [ ]:
model

Run:


In [ ]:
test_runner = TPZliteEstimator.make_stage(name="test_tpz", output="output_cosmodc2",
                                          model=model, **test_dict)

In [ ]:
%%time
results = test_runner.estimate(test_data)